In [1]:
#read the skills column of the person and put it in a list
import pandas as pd
import re
df = pd.read_csv("op1.csv")
s = df['Skills'].tolist()


#m is the total list, of all the skills all appended together
m = []
for i in s:
    str1 = i.split(',')
    for j in str1:
        m.append(j.lower())

#m is the total list, of all the skills all appended together, one person at a time
sentences = []
for i in s:
    temp = i.split(',')
    l = []
    for j in temp:
          l.append(j.lower())
    sentences.append(l)
    

#remove some crap entry
m = filter(lambda a: a != '', m)

In [2]:
#Counter to see how many times a skill appears to give initial weightage to a skill
from collections import Counter
counts = Counter(m)
countsfinal= []
for i in counts.items():
    l = []
    l.append(i[0])
    l.append(i[1])
    countsfinal.append(l)

In [3]:
#Normalize initial weights for each skill by using (1-val)/max and convert to dictionary format
countsfinal.sort(key = lambda x: x[1])
dict_weight = []
for i in countsfinal:
    init_weight = []
    init_weight.append(i[0])
    init_weight.append(1-(float(i[1])/countsfinal[-1][1]))
    dict_weight.append(init_weight)
d_init = dict(dict_weight)

In [4]:
print(d_init["machine learning"])

0.423469387755


In [5]:
#INITIAL WEIGHTAGES DONE
#WORD2VEC MODEL PART STARTS

In [6]:
#To see how much to increase the weightage of the skill we use wordToVec embeddings for each word 
#by correlating the vals value with the index of the skill.
import numpy
from gensim.models import Word2Vec
model = Word2Vec(sentences, min_count=10, iter = 20)

skills = []
for i in s:
    str1 = i.split(',')
    for j in str1:
        skills.append(j.lower())

vals = []
for i in skills:
    vals.append(numpy.linalg.norm(model[i])/100)

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
#Normalize the change in weights so that it is more noticeable in the final result
min1 = min(vals)
max1 = max(vals)

add_weight = []
for i in skills:
    a = []
    a.append(i)
    a.append((numpy.linalg.norm(model[i])/100 - min1)/(max1-min1) )
    add_weight.append(a)
dic_add = dict(add_weight)

In [8]:
dic_add['data analytics']

0.88528344515312918

In [9]:
sk_set = list(set(skills))
#sk_set
len(sk_set)
mat = []
for i in sk_set:
    mat.append(model[i])
    
#print(mat[:3])

In [10]:
len(model['machine learning'])

100

In [11]:
#END OF WORD2VEC
#PCA AND SIMILARITY MATCHING STARTS

In [12]:
from sklearn.decomposition import PCA
my_model = PCA(n_components=0.95, svd_solver='full') #this is as good as taking 95% variance
m1 = my_model.fit_transform(mat)

print(m1.shape)


(194, 15)


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
matrix = cosine_similarity(m1, m1)

In [14]:
removed = []
buckets = []
for i in range (0,matrix.shape[0]):
    l1 = []
    for j in range (i,matrix.shape[0]):
        if matrix[i][j] > 0.9 and j not in removed:
            l1.append(sk_set[j])
            removed.append(j)
    buckets.append(l1)  

In [15]:
buckets

[['nlp', 'keras', 'clustering', 'statistics'],
 ['network security',
  'virtualisation',
  'mips',
  'ipv4',
  'ipv6',
  'assembly level language',
  'http'],
 ['nosql'],
 ['kde', 'opensuse'],
 ['kafka', 'hbase', 'mapreduce', 'hive', 'ant'],
 ['kali-linux', 'emacs', 'make'],
 ['go', 'oop'],
 ['openstack', 'nova', 'dbaas'],
 [],
 ['xml', 'soa', 'eai', 'foaf'],
 ['makeitopen', 'fresco', 'react native'],
 ['arduino',
  'microprocessor',
  'pinguino',
  'nanode',
  'cypress',
  'microcontroller'],
 ['flask', 'html', 'jquery', 'nodejs', 'interfacing with cloud apis'],
 ['database-basked web sites',
  'angular2',
  'meanstack',
  'css',
  'cloud computing'],
 ['socket programming', 'web2.0', 'iot', 'wireless'],
 ['flash', 'reactjs', 'javascript'],
 ['selenium', 'visualisation', 'neural networks', 'classification'],
 ['shimmer', 'pathpicker', 'stetho'],
 ['angularjs', 'web semantics'],
 ['agile methodologies', 'sdlc', 'product management', 'program management'],
 [],
 [],
 ['sap crm',
  'inte

In [16]:
#In Case we take data scraped from linkedin or enter skills and Description separately

In [17]:
test_skills = ['machine learning','data analytics','hadoop','spark']
test_desc = 'Our project involves Machine Learning for Fast Optimal Search . It is implemented using Flask IT Policy Procedures Foundation'
accepted_buckets = []

#Finding those buckets that have the skills from test_skills
for i in test_skills:
    for j in range(0,len(buckets)):
        if i in buckets[j]:
            accepted_buckets.append(j)
            
#Appending the skills of the person with the related skills
#As a list of list so that final values of only appropriate set of skills are updated
final_pskills = []
for i in accepted_buckets:
    final_pskills.append(buckets[i])
    
#Check for 2 skills in same category, list gets added twice
final_pskills = [list(x) for x in set(tuple(x) for x in final_pskills)]

#Appending the skills of the person with the related skills as a list
test_skills = []
for i in accepted_buckets:
    test_skills.extend(buckets[i])
    
test_skills = list(set(test_skills))

#Converting provided description of project to lower case
test_desc = test_desc.lower()

#Append to a dictionay as key value pairs where key = skillname, value = its initial weightage
person_skill = []
for j in test_skills: 
    if j != '':
        p = []
        p.append(j)
        p.append(d_init[j])
        person_skill.append(p)
person_skill = dict(person_skill)

In [18]:
#These are the initial weights for all skills related to those given in test_skills field
person_skill

{'artificial intelligence': 0.41326530612244894,
 'bigdata': 0.44387755102040816,
 'chromium': 0.44047619047619047,
 'data analytics': 0.44557823129251706,
 'data mining': 0.44557823129251706,
 'database management': 0.45408163265306123,
 'digital ocean': 0.3928571428571429,
 'hadoop': 0.40476190476190477,
 'lens': 0.41836734693877553,
 'machine learning': 0.423469387755102,
 'spark': 0.4387755102040817,
 'tomcat': 0.435374149659864,
 'torch': 0.4302721088435374}

In [19]:
#Display final results using string matching and appropriate initial value and word2vec weights
#Append weights for all related skills if the skill is found in the description
for i in range (0,len(test_skills)):
    if re.search(test_skills[i],test_desc):
        for j in final_pskills:
            if test_skills[i] in j:
                for kk in j:
                    person_skill[kk] = person_skill[kk] + dic_add[kk]


In [20]:
#Displaying Updated skills beautifully
person_skill = sorted(person_skill.iteritems(), key=lambda i: i[1])
for i in person_skill:
    print(i)

('digital ocean', 0.3928571428571429)
('hadoop', 0.40476190476190477)
('lens', 0.41836734693877553)
('tomcat', 0.435374149659864)
('spark', 0.4387755102040817)
('bigdata', 0.44387755102040816)
('machine learning', 1.1191666502076822)
('torch', 1.1905071771114941)
('data mining', 1.2009438799067866)
('artificial intelligence', 1.2037065028596408)
('database management', 1.2092227715179447)
('chromium', 1.287414026469176)
('data analytics', 1.3308616764456462)


In [34]:
#If we want to read from PDF and do the same
#A person can give the skills he is looking for in a particular field in the test_skills field
#We parse the PDF of the person's resume as a text and look for the appropriate skill

In [26]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

In [101]:
#Used my resume for testing, Has nothing related to bigdata
#SO all the skills associated with spark will not have an increase in weightages
test_skills = ['labview','spark']
test_desc = convert ("Resume/01FB15ECS247.pdf")
accepted_buckets = []

#Finding those buckets that have the skills from test_skills
for i in test_skills:
    for j in range(0,len(buckets)):
        if i in buckets[j]:
            accepted_buckets.append(j)
            
#Appending the skills of the person with the related skills
#As a list of list so that final values of only appropriate set of skills are updated
final_pskills = []
for i in accepted_buckets:
    final_pskills.append(buckets[i])
    
#Check for 2 skills in same category, list gets added twice
final_pskills = [list(x) for x in set(tuple(x) for x in final_pskills)]

#Appending the skills of the person with the related skills as a list
test_skills = []
for i in accepted_buckets:
    test_skills.extend(buckets[i])

test_skills = list(set(test_skills))

#Converting provided description of project to lower case
test_desc = test_desc.lower()

#Append to a dictionay as key value pairs where key = skillname, value = its initial weightage
person_skill = []
for j in test_skills: 
    if j != '':
        p = []
        p.append(j)
        p.append(d_init[j])
        person_skill.append(p)
person_skill = dict(person_skill)

In [102]:
final_pskills

[['digital ocean', 'bigdata', 'spark', 'tomcat', 'lens'],
 ['labview', 'r', 'algorithms']]

In [103]:
person_skill

{'algorithms': 0.44387755102040816,
 'bigdata': 0.44387755102040816,
 'digital ocean': 0.3928571428571429,
 'labview': 0.4302721088435374,
 'lens': 0.41836734693877553,
 'r': 0.4319727891156463,
 'spark': 0.4387755102040817,
 'tomcat': 0.435374149659864}

In [104]:
#Display final results using string matching and appropriate initial value and word2vec weights
#Append weights for all related skills if the skill is found in the description
for i in range (0,len(test_skills)):
    if re.search(' '+test_skills[i]+ ' ',test_desc):
        for j in final_pskills:
            if test_skills[i] in j:
                for kk in j:
                    person_skill[kk] = person_skill[kk] + dic_add[kk]

In [105]:
#Displaying Updated skills beautifully
person_skill = sorted(person_skill.iteritems(), key=lambda i: i[1])
for i in person_skill:
    print(i)

('digital ocean', 0.3928571428571429)
('lens', 0.41836734693877553)
('tomcat', 0.435374149659864)
('spark', 0.4387755102040817)
('bigdata', 0.44387755102040816)
('r', 2.6645753222580546)
('labview', 2.6855884864971817)
('algorithms', 2.8468870841654139)
